Chạy `gcloud auth application-default login` để lấy application_default_credentials.json

Download thư viện cloud connector ở đây https://cloud.google.com/dataproc/docs/concepts/connectors/cloud-storage#clusters.

In [5]:
!gcloud auth application-default login

zsh:1: command not found: gcloud


In [10]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

credentials_location = "~/.config/gcloud/application_default_credentials.json"
conf = ( 
    SparkConf() 
        .setMaster('local[*]') 
        .setAppName('test') 
        .set('spark.jars', "./lib/gcs-connector-hadoop3-latest.jar") 
    #     # .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)
)

sc = SparkContext(conf=conf).getOrCreate()
hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
# hadoop_conf.set("fs.gs.auth.service.account.enable", "true")


spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

23/09/18 23:43:46 INFO SparkContext: Running Spark version 3.4.1
23/09/18 23:43:46 INFO ResourceUtils: ==============================================================
23/09/18 23:43:46 INFO ResourceUtils: No custom resources configured for spark.driver.
23/09/18 23:43:46 INFO ResourceUtils: ==============================================================
23/09/18 23:43:46 INFO SparkContext: Submitted application: test
23/09/18 23:43:46 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
23/09/18 23:43:46 INFO ResourceProfile: Limiting resource is cpu
23/09/18 23:43:46 INFO ResourceProfileManager: Added ResourceProfile id: 0
23/09/18 23:43:46 INFO SecurityManager: Changing view acls to: vince
23/09/18 23:43:46 INFO SecurityManager: Changing modi

In [11]:
data_path = "gs://aws-review-data/read/amazon_us_reviews-train-00000-of-00005.parquet"
df = spark.read.parquet(data_path)

23/09/18 23:46:34 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
23/09/18 23:46:34 INFO SharedState: Warehouse path is 'file:/Users/vince/protonx-DE/day_5/spark-warehouse'.
23/09/18 23:46:34 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://aws-review-data/read/amazon_us_reviews-train-00000-of-00005.parquet.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)

Py4JJavaError: An error occurred while calling o233.parquet.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlo

23/09/19 02:12:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3203624 ms exceeds timeout 120000 ms
23/09/19 02:12:26 WARN SparkContext: Killing executors is not supported by current scheduler.
23/09/19 02:12:30 INFO Executor: Told to re-register on heartbeat
23/09/19 02:12:30 INFO BlockManager: BlockManager BlockManagerId(driver, 192.168.1.4, 56469, None) re-registering with master
23/09/19 02:12:30 INFO BlockManagerMaster: Registering BlockManager BlockManagerId(driver, 192.168.1.4, 56469, None)
23/09/19 02:12:30 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.

In [3]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: long (nullable = true)
 |-- verified_purchase: long (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [5]:
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   0|                1|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [14]:
spark.stop()
